<a href="https://colab.research.google.com/github/Kulikov17/MLDS_CV/blob/main/cars_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>